In [11]:
##Libraries
import requests
import json
import pandas as pd

StatementMeta(, a7f7bf77-73fd-4453-8840-2c646f0ba451, 13, Finished, Available, Finished)

In [29]:


response = requests.get("https://open.er-api.com/v6/latest/USD")
byte_string  = response.content
json_string = byte_string.decode('utf-8')
data = json.loads(json_string)
json_output = json.dumps(data, indent=4)

# Print or use the JSON data
print(json_output)
#response_json = json.dumps(response.content)  

# Print the status code of the response.
print(response.status_code)
#print(response.content)

StatementMeta(, a7f7bf77-73fd-4453-8840-2c646f0ba451, 31, Finished, Available, Finished)

{
    "result": "success",
    "provider": "https://www.exchangerate-api.com",
    "documentation": "https://www.exchangerate-api.com/docs/free",
    "terms_of_use": "https://www.exchangerate-api.com/terms",
    "time_last_update_unix": 1725926551,
    "time_last_update_utc": "Tue, 10 Sep 2024 00:02:31 +0000",
    "time_next_update_unix": 1726013981,
    "time_next_update_utc": "Wed, 11 Sep 2024 00:19:41 +0000",
    "time_eol_unix": 0,
    "base_code": "USD",
    "rates": {
        "USD": 1,
        "AED": 3.6725,
        "AFN": 70.280485,
        "ALL": 90.124656,
        "AMD": 387.650264,
        "ANG": 1.79,
        "AOA": 929.440261,
        "ARS": 957.75,
        "AUD": 1.501447,
        "AWG": 1.79,
        "AZN": 1.701002,
        "BAM": 1.771531,
        "BBD": 2,
        "BDT": 119.554297,
        "BGN": 1.771485,
        "BHD": 0.376,
        "BIF": 2902.685027,
        "BMD": 1,
        "BND": 1.305812,
        "BOB": 6.920972,
        "BRL": 5.603765,
        "BSD": 1,
   

In [27]:

data = json.loads(json_output)


flattened_data = pd.json_normalize(data)


df = pd.DataFrame(flattened_data)


print(df)


StatementMeta(, a7f7bf77-73fd-4453-8840-2c646f0ba451, 29, Finished, Available, Finished)

    result                          provider  \
0  success  https://www.exchangerate-api.com   

                                documentation  \
0  https://www.exchangerate-api.com/docs/free   

                             terms_of_use  time_last_update_unix  \
0  https://www.exchangerate-api.com/terms             1725926551   

              time_last_update_utc  time_next_update_unix  \
0  Tue, 10 Sep 2024 00:02:31 +0000             1726013981   

              time_next_update_utc  time_eol_unix base_code  ...  rates.WST  \
0  Wed, 11 Sep 2024 00:19:41 +0000              0       USD  ...   2.694529   

    rates.XAF  rates.XCD  rates.XDR   rates.XOF   rates.XPF   rates.YER  \
0  594.145905        2.7   0.743463  594.145905  108.087297  250.397468   

   rates.ZAR  rates.ZMW  rates.ZWL  
0  17.875068  26.398155    13.9236  

[1 rows x 172 columns]


In [13]:
##Libraries
import requests
import json
import pandas as pd
from pyspark.sql.functions import col

##class to get the live currency from free API. 
base_url: str = "https://open.er-api.com/v6/latest"
class LiveCurrency:
    def __init__(
        self
    ) -> None:
        self.base_url = base_url
    def LiveCurrency_base(self, base: str) -> str:
        """Method def -
            Execute open API https://open.er-api.com/v6/latest using as parameter the currency
         Args:
             base (str): USD for dollar, GBP for Pounds
        Returns:
        Code response Success message
                       Fail message
        """
        try:
            url = (
                f"{self.base_url}/{base}"
            )
            #print (url)
            response = requests.get(url, timeout=120)
            ##remove the decode 'b if sucessfull
            if response.status_code==200:
                byte_string  = response.content
                json_string = byte_string.decode('utf-8')
                data = json.loads(json_string)
                json_output = json.dumps(data, indent=4)
                return json_output
            return response.raise_for_status
        except requests.exceptions.RequestException as req_err:
            return(f"Request error occurred: {req_err}")
    def LiveCurrency_table(self, base: str, table_base: str, append_overwrite: str) -> str:
        """Method def -
            Save live currency as table 
         Args:
             base (str): USD for dollar, GBP for Pounds
        Returns:
        Code response Success message
                       Fail message
        """
        try:
            ##flatten the output and save as table
            json_output= self.LiveCurrency_base(base)
            data = json.loads(json_output)
            flattened_data = pd.json_normalize(data)
            df = pd.DataFrame(flattened_data)
            
            ##making sure the column is using double as datatype and the name is only rates not rate.currency
            df = df.apply(lambda x: x.astype(float) if x.dtype == 'int64' else x)
            original_column_name = f"rates.{base}"
            df.rename(columns={original_column_name: 'rates'}, inplace=True)  
            spark_df = spark.createDataFrame(df)
            spark_df.write.format("delta").mode(append_overwrite).option("mergeSchema", "true").saveAsTable(table_base)

            return(f"{table_base} Created")
        except requests.exceptions.RequestException as req_err:
            return(f"Request error occurred: {req_err}")


StatementMeta(, 54fcd77a-fb2f-460a-b72c-21a1b8baccfd, 15, Finished, Available, Finished)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 47)

In [ ]:
client = LiveCurrency()

#response = client.LiveCurrency_base("USD")
#response
response = client.LiveCurrency_table("USD", "Live_Currency", "append")##_overwrite"
response 
response = client.LiveCurrency_table("GBP", "Live_Currency", "append")
response 
response = client.LiveCurrency_table("KYD", "Live_Currency", "append")
response 
response = client.LiveCurrency_table("JOD", "Live_Currency", "append")
response 
response = client.LiveCurrency_table("CHF", "Live_Currency", "append")
response 
response = client.LiveCurrency_table("EUR", "Live_Currency", "append")
response 
response = client.LiveCurrency_table("JPY", "Live_Currency", "append")
response 

In [11]:
df = spark.sql("SELECT * FROM testdata.livecurrency_table LIMIT 1000")
display(df)

StatementMeta(, 54fcd77a-fb2f-460a-b72c-21a1b8baccfd, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 30738c6e-7345-4424-90d1-066c63ba77a1)

In [10]:

client = LiveCurrency()

#response = client.LiveCurrency_base("USD")
#response
response = client.LiveCurrency_table("USD")
response ## management api scope

StatementMeta(, 54fcd77a-fb2f-460a-b72c-21a1b8baccfd, 12, Finished, Available, Finished)

https://open.er-api.com/v6/latest/USD
